In [1]:
!pip install selenium

In [2]:
from selenium.webdriver.support.ui import Select
import pandas as pd
from bs4 import BeautifulSoup
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import re
from dateutil.rrule import rrule, MONTHLY
from datetime import datetime
from google.cloud import storage
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
client = bigquery.Client()

In [4]:
def scraping(mesInicial,anioInicial,mesFinal,anioFinal):
    sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    def get_url(Mes,Anio):
        return "http://www.economia-sniim.gob.mx/AzucarMesPorDiaXregion.asp?Cons=D&prod=156&dqMesMes={Mes}&dqAnioMes={Anio}&Formato=Xls&submit=Ver+Resultados".format(Mes=Mes,Anio=Anio)
    def months(start_month, start_year, end_month, end_year):
        start = datetime(start_year, start_month, 1)
        end = datetime(end_year, end_month, 1)
        return [(d.month, d.year) for d in rrule(MONTHLY, dtstart=start, until=end)]
    df2=pd.DataFrame()
    a= months(mesInicial,anioInicial,mesFinal,anioFinal)
    for i in range(0,len(a),1):
        data = requests.get(get_url(a[i][0],a[i][1])).text
        anio=a[i][1]
        soup = BeautifulSoup(data, 'html.parser')
        table = soup.find_all('table')
        for row in table[17]:
            text = row.find_all('td')
        list_header = [] 
        header = table[17].find("tr") 
        for items in header: 
            try: 
                list_header.append(items.get_text()) 
            except: 
                continue
        data = [] 
        HTML_data = table[17].find_all("tr")[1:]
        for element in HTML_data: 
            sub_data = [] 
            for sub_element in element: 
                try: 
                    sub_data.append(sub_element.get_text()) 
                except: 
                    continue
            data.append(sub_data) 
        df = pd.DataFrame(data = data[3:])
        df.columns=df.iloc[0,:].values
        df=df.loc[1:,:]
        df=df.loc[df["Centros de Distribución"].str.contains('Centro') != True]
        df=df.loc[df["Centros de Distribución"].str.contains('Occidente') != True]
        df=df.loc[df["Centros de Distribución"].str.contains('Noroeste') != True]
        df=df.loc[df["Centros de Distribución"].str.contains('Noreste') != True]
        df=df.loc[df["Centros de Distribución"].str.contains('Golfo') != True]
        df=df.loc[df["Centros de Distribución"].str.contains('Sureste') != True]  	
        df=df.loc[df["Centros de Distribución"].str.contains('Fuente: Precios:') != True]
        df=df.loc[df["Centros de Distribución"] != ""] 
        df.drop(["PromMes 1/"], axis=1,inplace=True)
        df.set_index('Centros de Distribución',inplace=True)
        df = df.rename(columns=lambda x: re.sub('\xa0\xa0','  ',str(x)))
        df=df.dropna(1)
        df=df.T
        df.reset_index(inplace=True)
        df.rename(columns={'index':'fecha'},inplace=True)
        df = df.rename(columns=lambda x: re.sub('\xa0',' ',x))
        col=list(df.columns)
        col.remove("fecha")
        for i in range(len(col)):
          df1=df[["fecha", str(col[i]) ]] 
          df1["Centro_distribucion"]= col[i]
          df1["year"]= anio
          df1.rename(columns={col[i]:'valor'},inplace=True)
          df1['month'] = df1['fecha'].str[-3:]
          df1['day'] = df1['fecha'].str[-6:-4]
          df2= pd.concat([df2,df1])
        di = {"Ene": 1, "Feb": 2,"Mar": 3,"Abr": 4,"May": 5,"Jun": 6,"Jul": 7,"Ago":8,"Sep":9,"Oct":10,"Nov":11,"Dic":12}
        df2=df2.replace({"month": di})
        df2['date_time']=pd.to_datetime(df2[['year','month','day']])
    df2=df2[["date_time","Centro_distribucion","valor"]]
    df2=df2[~df2['valor'].isna()]
    #df2["Centro_distribucion"] = df2["Centro_distribucion"].astype('string') 
    df2['valor']=df2['valor'].str.replace(",","")
    df2["valor"] = pd.to_numeric(df2["valor"],errors = 'coerce')
    return df2

In [ ]:
#INICIANDO DESCARGA DE LA DATA
now = datetime.now()
df2=scraping(10,2010,now.month,now.year)   #scraping(10,2000,6,2022) 

# LECTURA DE DATOS EN BQ
PROJECT_ID = 'timeseries-387916'

# ELIMINAR DUPLICADOS
result=df2.drop_duplicates()

# ACTUALIZACION
result.to_gbq(f'{PROJECT_ID}.SNIIM.scraping', if_exists='replace', project_id=f"{PROJECT_ID}")  #timeseries-387916.SNIIM.scraping
print(df2['date_time'].max())
return "OK"

/var/tmp/ipykernel_190339/3026616073.py:53: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  df=df.dropna(1)
/var/tmp/ipykernel_190339/3026616073.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Centro_distribucion"]= col[i]
/var/tmp/ipykernel_190339/3026616073.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["year"]= anio
/var/tmp/ipykernel_190339/3026616073.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [ ]:
df2['date_time'].max()

In [ ]:
PROJECT_ID = 'timeseries-387916'
result=df2.drop_duplicates()
result.to_gbq(f'{PROJECT_ID}.SNIIM.scraping', if_exists='replace', project_id=f"{PROJECT_ID}")  #timeseries-387916.SNIIM.scraping

In [ ]:
#INICIANDO DESCARGA DE LA DATA
now = datetime.now()
now.month, now.year
df2=scraping(now.month,now.year,now.month,now.year)   #scraping(10,2000,6,2022) 
print("xd")
# LECTURA DE DATOS EN BQ
PROJECT_ID = 'timeseries-387916' #timeseries-387916
# data_bq = client.query(f""" SELECT date_time,Centro_distribucion ,valor FROM {PROJECT_ID}.raw.SNIM_PRECIOS_REFERENCIA""").to_dataframe()   #raw.SNIM_PRECIOS_REFERENCIA
# data_bq  = data_bq.astype({"Centro_distribucion":'string'})
# data_bq  = data_bq.astype({"date_time":'string'})
df2  = df2.astype({"date_time":'string'})
df2  = df2.astype({"Centro_distribucion":'string'})
# COMBINAR DATA
frames = [data_bq, df2]
result = pd.concat(frames)
# ELIMINAR DUPLICADOS
result=result.drop_duplicates()
# In[23]:
# ACTUALIZACION

##result.to_gbq(f'{PROJECT_ID}.raw.SNIM_PRECIOS_REFERENCIA', if_exists='replace', project_id=f"{PROJECT_ID}") 
return "OK"